# Rich feature hierarchies for accurate object detection and semantic segmentation (R-CNN)  
- 논문 리뷰

## Abstract  
- object detection은 지난 몇 년 간 발전이 정체되어 있었다.  
- 본 논문은 VOC 2012 data set에서 전보다 30% 이상의 성능을 높인 알고리즘을 제안한다.  
- 이때 2가지 인사이트를 결합한다.  
    - 1. object를 localize하고 segment하기 위해 region proposal에 CNN을 적용  
    - 2. 라벨링된 훈련 데이터가 부족할 때 fine-tuning을 거친 지도 사전 학습을 이용해 성능 향상  


## Introduction  
- 처음으로 CNN이 PASCAL VOC data set에서 높은 성능의 object detection을 이끌 수 있단 것을 보여준다.  
- 본 논문에서는 다음 2가지에 집중했다.  
    - 1. 깊은 네트워크로 obejct를 localizing하는 것  
    - 2. 적은 양의 annotated된 detection data를 가지고 고성능의 모델을 훈련하는 것  
    
<img src = "https://images.velog.io/images/skhim520/post/fa1ea5cf-6064-490d-ad52-dd6d5d00bbfe/image.png">  

- 본 논문은 "region을 이용한 인식" 패러다임을 이용해 연산을 수행함  
- 그래서 CNN localization 문제를 해결함  
- 이 논문의 method는 위 그림과 같이 4단계를 거친다  
    - step1. 이미지를 입력  
    - step2. 2000개의 region proposal을 생성  
    - step3. 각 region proposal을 CNN에 입력해 feature vector를 얻음  
    - step4. linear SVM을 이용해 각 region proposal이 어떤 클래스인지 분류  
    
- 이때 각 region proposal을 CNN에 입력하기 위해 고정된 input size로 warp을 진행한다  
- 이를 통해 R-CNN의 성능은 이전의 OverFeat을 능가함 (24.3% $\rightarrow$ 31.4%)  
- fine-tuning을 한 결과 mAP가 8% 정도 개선됨  
- 또한 DPM 모델과 비교해 성능이 33% 이상 끌어올려 54%의 mAP를 달성함

## Object detection with R-CNN  
- R-CNN의 시스템은 3가지 모듈로 구성됨  
    - 1. region proposal  
    - 2. CNN에 입력 후 고정된 길이의 feature vector 얻기  
    - 3. linear SVM을 통해 클래스 분류  
    
### Module design  
**Region proposals**  
- selective search를 이용해 region을 추출  

**Feature extraction**  
- 각 region proposal에서 4096차원의 feature vector를 추출  
- 추출된 feature들은 227x227 RGB 이미지에서 5개의 convolution layer와 2개의 fully connected layer를 통해 계산됨  
- 이때 CNN에 입력하기 위해 input size에 맞게 변형함  

<img src = "https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=http%3A%2F%2Fcfile5.uf.tistory.com%2Fimage%2F99F02F505CD83FC0093ACE">  

- 후보 region의 사이즈나 가로 세로 비율에 상관없이 모든 bounding box를 warp한다  
- 이때 warpping을 하기 전에 타이트한 bounding box가 있다면 그 주변을 $p$개의 픽셀만큼 채워 warpping을 진행  
- 논문에서는 $p=16$일 때가 성능이 가장 좋게 나온다고 알려짐  

### Test-time detection  
- 각 proposal을 warpping을 하고 feature를 얻기 위해 CNN을 통해 학습을 시킨다  
- 그러면 linear SVM을 통과시켜 각 클래스에 대한 score를 얻을 수 있다  
- 이때 NMS(non-maximum suppresion)를 이용해 IoU와 threshold보다 낮은 score는 제거한다  
- 즉, score가 높은 bounding box만 남김  

**Run-time analysis**  
- 두 가지 성질이 detection 수행을 효율적이게 만든다  
    - 1. 모든 CNN 파라미터는 모든 카테고리에 걸쳐 공유됨  
    - 2. CNN에서 계산된 feature vector는 다른 접근법과 비교해 낮은 차원을 가짐  
    
### Training  
**Supervised pre-training**  
- ILSVRC2012 data set에서 사전 훈련된 CNN을 이용  

**Domain-specific fine-tuning**  
- CNN을 detection task와 새로운 domain에 적용하기 위해 warped region proposal만을 이용해 SGD로 파라미터들을 훈련시킴  
- CNN의 classification layer를 $N+1$개로 수정   
- $N$: 클래스 갯수, $+1$: background  
- 이 외에는 acchitecture의 수정 사항이 없음  
- IoU가 0.5 이상이면 positive sample, 아니면 negative sample로 정의  
- optimizer는 SGD, learning rate는 0.001로 설정  
- 128개의 mini-batch를 위해 SGD iteration마다 positive sample 32개, background 96개 사용  

**Object category classifiers**  
- positive sample: ground truth bounding box  
- negative sample: IoU가 0.3 미만인 경우

**속도와 정확도**  
- R-CNN은 이미지 하나 당 GPU에서는 13초, CPU에서는 54초가 걸린다  
- Selective Search를 통해 찾은 region proposal들을 CNN에 통과시키는 것에서 대부분의 시간이 소요된다  
- 정확도의 경우 Pascal VOC 2010을 기준으로 53.7%를 기록했다  
- 비록 좋은 정확도는 아니지만 이 논문은 Object Detection 분야의 발전을 이끌었다